En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# Librerias y path

In [10]:
import emoji
from collections import Counter

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
from pandas import json_normalize

#file_path = "farmers-protest-tweets-2021-2-4.json"
file_path2 = "C:\\Users\\JuanJo\\Desktop\\LATAM\\Challenge\\farmers-protest-tweets-2021-2-4.json"



# Dataframes

### df

Tweet original

In [2]:
df0=pd.read_json(file_path2, lines=True)
df=df0.dropna(axis=1, how='all')
df['fecha'] = df['date'].dt.strftime('%Y-%m-%d')

### df_users

detalles de cada user

In [3]:
df_users = json_normalize(df['user'])
df_users['fecha'] = df['fecha'].values

In [4]:
df_acotado2=df[['id','media']]
df_acotado3=df[['id','quotedTweet']]
df_acotado4=df[['id','mentionedUsers']]

In [82]:
#dato=df_acotado2.iloc[0].media

### df_media
detalles de informacion tipo media

In [5]:
df_expandido = df_acotado2.explode('media')
df_media = json_normalize(df_expandido['media'])
df_media['id_prim'] = df_expandido['id'].values
df_media=df_media.dropna(axis=1, how='all')
df_media2 = df_media[df_media['type'].notnull()]

### df_quotedTweet

reTweetos

In [6]:
df_quotedTweet = json_normalize(df['quotedTweet'])
df_quotedTweet['id_prim'] = df_acotado3['id'].values
df_quotedTweet2 = df_quotedTweet[df_quotedTweet['id'].notnull()]
df_quotedTweet3=df_quotedTweet2.dropna(axis=1, how='all')
df_quotedTweet3['fecha'] = pd.to_datetime(df_quotedTweet3['date']).dt.strftime('%Y-%m-%d')


C:\Users\JuanJo\AppData\Local\Temp\ipykernel_16812\1096978238.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quotedTweet3['fecha'] = pd.to_datetime(df_quotedTweet3['date']).dt.strftime('%Y-%m-%d')


### df_mentionedUsers

Usuarios mencionados en tweet original

In [7]:
df_expandido = df_acotado4.explode('mentionedUsers')
df_mentionedUsers = json_normalize(df_expandido['mentionedUsers'])
df_mentionedUsers['id_prim'] = df_expandido['id'].values
df_mentionedUsers=df_mentionedUsers.dropna(axis=1, how='all')
df_mentionedUsers2 = df_mentionedUsers[df_mentionedUsers['id'].notnull()]

### df_mentionedUsers_quot

usuarios mencionados en retweeteos

In [8]:
#df_quotedTweet3_acotado=df_quotedTweet3[['id','id_prim','user.username','user.id','mentionedUsers']]
df_quotedTweet3_acotado=df_quotedTweet3[['id','mentionedUsers']]
df_quotedTweet3_acotado = df_quotedTweet3_acotado[df_quotedTweet3_acotado['mentionedUsers'].notnull()]

df_expandido = df_quotedTweet3_acotado.explode('mentionedUsers')
df_mentionedUsers_quot = json_normalize(df_expandido['mentionedUsers'])
df_mentionedUsers_quot['id_second'] = df_expandido['id'].values
df_mentionedUsers_quot=df_mentionedUsers_quot.dropna(axis=1, how='all')

#dato=df_quotedTweet3_acotado.iloc[0].mentionedUsers

## Pruebas seguimiento de datos

In [9]:
dato0_df=df_quotedTweet3.loc[0]
dato0_idprim=df_quotedTweet3.iloc[0].id_prim
dato0_id=int(df_quotedTweet3.iloc[0].id)
dato0_2_comp=dato0_df.mentionedUsers

dato2_df=df[df['id']==dato0_idprim].loc[0]


dato2_2_df=df_mentionedUsers2[df_mentionedUsers2['id_prim']==dato0_idprim]
dato0_2_df=df_mentionedUsers_quot[df_mentionedUsers_quot['id_second']==dato0_id]



# Pregunta 1

##### a- top10 fechas con más tweets
###### a- Considerando los retweets un tweet por si mismo se hace la suma de los ids por fechas de los tweet y los retweets y se rankea fechas principales.

##### b- usuarios con más publicaciones dichos dias
###### b- Con estas fechas se cuentan los usarios que emitieron tweets y retweets y se obtiene el maximo usuario para estas 10 fechas.

In [142]:
df_agrup = df.groupby(df['fecha'])['id'].count()
#print(len(df[df['fecha']=='2021-02-23'])) ##comprobacion
df_retw_agrup = df_quotedTweet3.groupby(df_quotedTweet3['fecha'])['id'].count()

In [126]:
# Fusionar los DataFrames en función de las fechas
df_merged = pd.merge(df_agrup, df_retw_agrup, on='fecha', how='outer', suffixes=('_dftweets', '_dfretweets'))
df_merged['cantidad_total'] = df_merged['id_dftweets'].fillna(0) + df_merged['id_dfretweets'].fillna(0)

df_merged_sorted = df_merged.sort_values(by='cantidad_total', ascending=False)
top_10_fechas = df_merged_sorted.head(10)


#### Caso especifico para una fecha

In [ ]:
df_user_agrup_0 = df_users.groupby(df_users[df_users['fecha']==top_10_fechas.index[0]]['username'])['id'].count()#.sum() #calza
#print(len(df_users[df_users['fecha']==top_10_fechas.index[0]]))


df_retwet_agrup_0 = df_quotedTweet3.groupby(df_quotedTweet3[df_quotedTweet3['fecha']==top_10_fechas.index[0]]['user.username'])['user.id'].count()#.sum() #calza
#df_retwet_agrup_0=df_retwet_agrup_0.rename({'user.id':'id'})

df_retwet_agrup_0.rename("id",inplace=True)
df_retwet_agrup_0.index.name="username"
#df_retwet_agrup_0.rename("username",axis='index',inplace=True)
#print(len(df_quotedTweet3[df_quotedTweet3['fecha']==top_10_fechas.index[0]]))

In [ ]:

df_merged_users = pd.merge(df_user_agrup_0, df_retwet_agrup_0, on='username', how='outer', suffixes=('_dftweets', '_dfretweets'))
df_merged_users ['cantidad_total'] = df_merged_users ['id_dftweets'].fillna(0) + df_merged_users ['id_dfretweets'].fillna(0)

df_merged_users_sorted = df_merged_users .sort_values(by='cantidad_total', ascending=False)
top_1 = df_merged_users_sorted.head(1)


### Ciclo for para respuesta

In [134]:
P1_data = {'Fecha': [], 'Usuario':[]}
P1 = pd.DataFrame(data=P1_data)


for i in top_10_fechas.index:

    df_user_agrup_0 = df_users.groupby(df_users[df_users['fecha']==i]['username'])['id'].count()
    df_retwet_agrup_0 = df_quotedTweet3.groupby(df_quotedTweet3[df_quotedTweet3['fecha']==top_10_fechas.index[0]]['user.username'])['user.id'].count()
    df_retwet_agrup_0.rename("id",inplace=True)
    df_retwet_agrup_0.index.name="username"
    df_merged_users = pd.merge(df_user_agrup_0, df_retwet_agrup_0, on='username', how='outer', suffixes=('_dftweets', '_dfretweets'))
    df_merged_users ['cantidad_total'] = df_merged_users ['id_dftweets'].fillna(0) + df_merged_users ['id_dfretweets'].fillna(0)
    df_merged_users_sorted = df_merged_users .sort_values(by='cantidad_total', ascending=False)
    top_1 = df_merged_users_sorted.head(1)
    P1.loc[len(P1)] = np.array([i,top_1.index[0]])

In [137]:
P1
P1['Fecha'] = pd.to_datetime(P1['Fecha'])

In [160]:
output_list_P1 = [(fecha.date(), usuario) for fecha, usuario in zip(P1['Fecha'], P1['Usuario'])]
print(output_list_P1)

[(datetime.date(2021, 2, 12), 'RaviSinghKA'), (datetime.date(2021, 2, 13), 'RaviSinghKA'), (datetime.date(2021, 2, 17), 'RaviSinghKA'), (datetime.date(2021, 2, 14), 'RaviSinghKA'), (datetime.date(2021, 2, 16), 'RaviSinghKA'), (datetime.date(2021, 2, 18), 'RaviSinghKA'), (datetime.date(2021, 2, 15), 'RaviSinghKA'), (datetime.date(2021, 2, 19), 'Preetm91'), (datetime.date(2021, 2, 23), 'RaviSinghKA'), (datetime.date(2021, 2, 20), 'RaviSinghKA')]


# Pregunta 2

##### a- top10 emojis más usados
###### a- Considerando los retweets un tweet por si mismo se busca en cada content la existencia de emojis y se cuentan.

##### b- conteo de estos emojis
###### b- Se hace la suma general de todo por emoji y se rankea

In [16]:
emojis = []
for i in df['content']:
    emojis += [c for c in i if c in emoji.EMOJI_DATA]
conteo_emojis = Counter(emojis)
df_emojis_twet = pd.DataFrame(list(conteo_emojis.items()), columns=['emoji', 'cantidad'])
df_emojis_twet.set_index('emoji', inplace=True)
#print(df_emojis_twet)

emojis_ret = []
for i in df_quotedTweet3['content']:
    emojis_ret += [c for c in i if c in emoji.EMOJI_DATA]
conteo_emojis = Counter(emojis_ret)
df_emojis_retwet = pd.DataFrame(list(conteo_emojis.items()), columns=['emoji', 'cantidad'])
df_emojis_retwet.set_index('emoji', inplace=True)
#print(df_emojis_retwet)

In [18]:
df_merged_emoji = pd.merge(df_emojis_twet, df_emojis_retwet, on='emoji', how='outer', suffixes=('_dftweets', '_dfretweets'))
df_merged_emoji ['cantidad_total'] = df_merged_emoji ['cantidad_dftweets'].fillna(0) + df_merged_emoji ['cantidad_dfretweets'].fillna(0)

df_merged_emoji_sorted = df_merged_emoji .sort_values(by='cantidad_total', ascending=False)
top_10_emoji = df_merged_emoji_sorted.head(10)


In [19]:
top_10_emoji=top_10_emoji.reset_index()

In [20]:
P2=top_10_emoji[['emoji','cantidad_total']]
P2['cantidad_total'] = P2['cantidad_total'].astype(int)

C:\Users\JuanJo\AppData\Local\Temp\ipykernel_16812\3427506781.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  P2['cantidad_total'] = P2['cantidad_total'].astype(int)


In [21]:
output_list_P2 = [(emo, int(cantidad)) for emo, cantidad in zip(P2['emoji'], P2['cantidad_total'])]
print(output_list_P2)

[('🙏', 8916), ('😂', 4067), ('🚜', 3274), ('✊', 3046), ('🏻', 2696), ('🌾', 2652), ('❤', 2263), ('🤣', 2110), ('👉', 1895), ('👇', 1691)]


# Pregunta 3

##### a- top10 historico users con más menciones
###### a- Considerando los retweets un tweet por si mismo se busca en en los dataframes de usarios mencionados y usuarios mencionados en los retweets cada aparicion de usuarios.

##### b- conteo de estas menciones
###### b- Se hace la suma general de todo por usuario y se rankea

In [143]:
df_agrup_hist = df_mentionedUsers2.groupby(df_mentionedUsers2['username'])['id'].count()
#print(len(df[df['fecha']=='2021-02-23'])) ##comprobacion
df_retw_hist_agrup = df_mentionedUsers_quot.groupby(df_mentionedUsers_quot['username'])['id'].count()

In [151]:
df_merged_menc = pd.merge(df_agrup_hist, df_retw_hist_agrup, on='username', how='outer', suffixes=('_dftweets', '_dfretweets'))
df_merged_menc['cantidad_total'] = df_merged_menc['id_dftweets'].fillna(0) + df_merged_menc['id_dfretweets'].fillna(0)

df_merged_menc_sorted = df_merged_menc.sort_values(by='cantidad_total', ascending=False)
top_10_menciones = df_merged_menc_sorted.head(10)


In [152]:
top_10_menciones=top_10_menciones.reset_index()

In [153]:
P3=top_10_menciones[['username','cantidad_total']]

In [156]:
P3
P3['cantidad_total'] = P3['cantidad_total'].astype(int)

C:\Users\JuanJo\AppData\Local\Temp\ipykernel_7932\670438264.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  P3['cantidad_total'] = P3['cantidad_total'].astype(int)


In [159]:
output_list_P3 = [(username, int(cantidad)) for username, cantidad in zip(P3['username'], P3['cantidad_total'])]
print(output_list_P3)

[('narendramodi', 2623), ('Kisanektamorcha', 2045), ('RakeshTikaitBKU', 1848), ('PMOIndia', 1560), ('GretaThunberg', 1274), ('RahulGandhi', 1252), ('rihanna', 1142), ('DelhiPolice', 1134), ('RaviSinghKA', 1127), ('UNHumanRights', 1057)]
